# Scripted spectra

In [ ]:
using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, Interpolations
using JLD, HDF5, Printf
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

using PyCall
np = pyimport("numpy")

PATH = "/home/ubuntu/Archives/"
PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
N = 9
pattern = r"random_SK_instance_N_9_seed_(\d+)\.h5"

# N = 11
# pattern = r"random_SK_instance_N_11_seed_(\d+)\.h5"

# N = 13
# pattern = r"random_SK_instance_N_13_seed_(\d+)\.h5"

# N = 15
# pattern = r"random_SK_instance_N_15_seed_(\d+)\.h5"

# N = 17
# pattern = r"random_SK_instance_N_17_seed_(\d+)\.h5"

# N = 19
# pattern = r"random_SK_instance_N_19_seed_(\d+)\.h5"

subdir = "small_gaps"
subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name)
filter!(x -> !occursin("results", x), instance_names);

In [ ]:
# T_final = 32000.
T_final = 32768.
tol = 1e-6
# τ_final = 2000.
τ_final = 2048.

npts_diag = 16
T_diags = T_final .* range(0.5, 1.0, npts_diag+1)
@sprintf("spectra_T_final_%i_tau_final_%i/T_%0.3f/omegas", T_final, τ_final, T_diags[1] / T_final)

In [ ]:
all_mf_Es = []
all_exact_E0s = []
all_exact_E1s = []
all_exact_E2s = []
all_ωs = []
spec_sum = []

ordered_seeds = []
missing_seeds = []
seed_to_energy_diff = Dict()

# num_plots = 100
counter = 0
for instance_name in instance_names#[1:num_plots]
    try
        seed = match(pattern, instance_name)[1]    

        # mean-field solution and energy
        J_mat = h5read(folder_name * instance_name, "J")
        mf_problem = Problem(0, J_mat)

        mf_sol = h5read(folder_name * "results_" * instance_name, @sprintf("mean_field_sol_T_final_%.0f_tol_1e%.0f", T_final, log10(tol)))
        sigma_star = sign.(mf_sol)
        h = mf_problem.local_fields
        J = mf_problem.couplings
        E_star = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])
        push!(all_mf_Es, E_star)
        
        # gap and spectra
        λ = h5read(folder_name * instance_name, "exact_ARPACK_LM_eigvals")
        push!(all_exact_E0s, λ[1, end])
        push!(all_exact_E1s, λ[2, end])
        push!(all_exact_E2s, λ[3, end])

        # println(seed, ": " , E_star .- λ[1, end])

        gap = λ[2, :] .- λ[1, :]    
        # println(seed, ": " , 1/minimum(gap), ", ", E_star .- λ[1, end])
        seed_to_energy_diff[seed] = (E_star .- λ[1, end], E_star .- λ[1, end] / abs(λ[2, end] .- λ[1, end]))

        if isapprox(minimum(gap), 0.0, atol=0.002)
        # if isapprox(minimum(gap), 0.01, atol=0.002)
            counter += 1
        end

        exact_times = range(0, 1, 33)
        gap_idx = findfirst(x -> x == minimum(gap), gap)
        gaploc = exact_times[gap_idx] 
        
        # T_diags = T_final .* [gaploc - 0.1, gaploc - 0.05, gaploc]

        gap_idx = gap_idx - 16 # we use only the interval [0.5, 1.0] with 16+1 points

        # for k in gap_idx:gap_idx
        # for k in [gap_idx]
        for k in [1]
            push!(all_ωs, h5read(folder_name * "results_" * instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[k] / T_final)))
            push!(spec_sum, h5read(folder_name * "results_" * instance_name, @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/data", T_final, τ_final, T_diags[k] / T_final)))
        end 
        push!(ordered_seeds, seed)  
    catch
        seed = match(pattern, instance_name)[1]
        push!(missing_seeds, seed)
        # printstyled(seed, "\n", color=:red)
    end
end

In [ ]:
sort(seed_to_energy_diff |> collect, by=x->x[2][1])

In [ ]:
missing_seeds |> println

In [ ]:
num_plots = length(spec_sum)

In [ ]:
println(counter)
println(counter/num_plots)

In [ ]:
ΔE = all_mf_Es .- all_exact_E0s
ΔE1 = all_mf_Es .- all_exact_E1s
ΔE2 = all_mf_Es .- all_exact_E2s
mean(ΔE) |> println
mean(ΔE1) |> println
mean(ΔE2) |> println

In [ ]:
findall(x -> x == 0, ΔE) |> length |> println
findall(x -> isapprox(x, 0.0, atol=1e-1), ΔE) |> length |> println
findall(x -> isapprox(x, 0.0, atol=1e-1), ΔE1) |> length |> println
findall(x -> isapprox(x, 0.0, atol=1e-1), ΔE2) |> length |> println

In [ ]:
findall(x -> isapprox(x, 0.0, atol=1e-13), ΔE) |> length |> println

In [ ]:
ΔE = (all_mf_Es .- all_exact_E0s) ./ abs.(all_exact_E0s .- all_exact_E1s)
ΔE1 = (all_mf_Es .- all_exact_E1s) ./ abs.(all_exact_E0s .- all_exact_E1s)
ΔE2 = (all_mf_Es .- all_exact_E2s) ./ abs.(all_exact_E1s .- all_exact_E2s);

# ΔE = (all_mf_Es .- all_exact_E0s) ./ abs.(all_exact_E0s)
# ΔE1 = (all_mf_Es .- all_exact_E1s) ./ abs.(all_exact_E1s)
# ΔE2 = (all_mf_Es .- all_exact_E2s) ./ abs.(all_exact_E2s);

In [ ]:
num_bins = 200
num_bins = 20
cutoff = 200

# counts, bins = np.histogram(log.(abs.(ΔE) .+ 1.0im), bins=num_bins)
# counts, bins = np.histogram(ΔE, bins=num_bins)
counts, bins = np.histogram(sort(ΔE)[1:num_plots-cutoff], bins=num_bins)
bins = real.(bins)

# counts_1, bins_1 = np.histogram(log.(abs.(ΔE1) .+ 1.0im), bins=num_bins)
# counts_1, bins_1 = np.histogram(ΔE1, bins=num_bins)
counts_1, bins_1 = np.histogram(sort(ΔE1)[1:num_plots-cutoff], bins=num_bins)
bins_1 = real.(bins_1)

# PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["midnightblue", "seagreen", "darkcyan", "slategray"]))
PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))
# PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["#B7293F", "#2D5FAA", "#438E6A", "#F7BD2E", "#F16C37"]))

figure(figsize=(4, 3))
hist(bins[1:end-1], bins, weights=counts/length(ΔE), alpha=0.9, label="\$E_0\$")#, label="Ground")
hist(bins_1[1:end-1], bins_1, weights=counts_1/length(ΔE1), alpha=0.9, label="\$E_1\$")#, label="First excited")
legend(frameon=false)
# xlim(-0.02, 0.04)
# xlim(-1, 9)
# ylim(0, 1.0)
# xlabel("\$\\ln(E_* - E_i)\$")
# xlabel("\$E_* - E_i\$")
# xlabel("\$(E_* - E_i) / \\left|E_i\\right|\$")
xlabel("\$(E_* - E_i) / \\left|E_0 - E_1\\right|\$")
ylabel("\$P(E_*)\$")
tight_layout()
# savefig(PLOT_PATH * @sprintf("mf_sol_hist_N_%i.pdf", N), dpi=256, bbox_inches="tight")

In [ ]:
# map data to functions
spectral_sum_fs = []
for (k, (ωs, spec)) in enumerate(zip(all_ωs, spec_sum))
    push!(spectral_sum_fs, linear_interpolation(ωs, spec, extrapolation_bc=Line()))
end

In [ ]:
xlim_vals = (-0.05, 0.4)
new_ωs = range(xlim_vals..., 2001)
spectral_average = mean([map(spectral_sum_fs[k], new_ωs) for k in 1:length(spectral_sum_fs)]);

In [ ]:
tmp_ωs = new_ωs
tmp_spectral_average = spectral_average;

In [ ]:
figure(figsize=(4, 3))
subplot(111)
axvline(0.01, ls="--") 
plot(new_ωs, spectral_average, "-k", ms=2)   
# xlim(xlim_vals...)
# xlim(-0.01, 0.04)
# ylim(-0., 0.1)
ylim(-0., 0.05)
xlabel("\$\\nu\$")
ylabel("Ensemble-averaged spectrum", fontsize=12)
tight_layout()

In [ ]:
figure(figsize=(4, 3))
subplot(111)
axvline(0.01, ls="--") 
plot(new_ωs, spectral_average, "-k", ms=2)   
# xlim(xlim_vals...)
# xlim(-0.01, 0.04)
ylim(-0., 0.1)
ylim(-0., 0.05)
xlabel("\$\\nu\$")
ylabel("Ensemble-averaged spectrum", fontsize=12)
tight_layout()

## Testing

In [ ]:
figure(figsize=(4, 3))
subplot(111)
axvline(0.01, ls="--") 
plot(new_ωs, spectral_average .- tmp_spectral_average, "-k", ms=2)   
# xlim(xlim_vals...)
# xlim(-0.01, 0.04)
ylim(-0., 0.02)
xlabel("\$\\nu\$")
ylabel("Ensemble-averaged spectrum", fontsize=12)
tight_layout()

In [ ]:
# for idx in findall(x -> isapprox(x, 0.0, atol=8e-3), collect(new_ωs))
#     # println(new_ωs[idx])
#     spectral_average[idx] = 0.0
# end

In [ ]:
num_bins = 100
# counts_1, bins_1 = np.histogram(log.(abs.(ΔE1) .+ 1.0im), bins=num_bins)
counts_2, bins_2 = np.histogram(ΔE2, bins=num_bins)
# bins_1 = real.(bins_1)

# PyPlot.rc("axes", prop_cycle=PyPlot.plt.cycler(color=["slategray", "midnightblue", "seagreen", "darkcyan"]))

figure(figsize=(4, 3))
hist(bins_2[1:end-1], bins_2, weights=counts_1/length(ΔE2))
# legend(frameon=false)
# xlim(-0.1, 0.2)
ylim(0, 1.)
xlabel("\$\\ln(E_* - E_0)\$")
ylabel("\$P(E_*)\$")
tight_layout()
# savefig(@sprintf("../plots/Histogram_MAX2SAT_L_%i.pdf", L))

In [ ]:
zip([1], [2], [3]) |> collect

In [ ]:
full_data = collect(zip(ordered_seeds, all_ωs, spec_sum))
start_idx = 1200
plot_data = enumerate(full_data[start_idx:start_idx + 50])

xlim_vals = (-0.02, 0.02)
fig, axs = subplots(figsize=(5, 40), nrows=length(plot_data), ncols=1)

for (k, (seed, ωs, spec)) in plot_data
    ax = axs[k]     
    ax.plot(ωs, spec, "-", label=seed)

    ax.set_xlim(xlim_vals...)
    ax.set_ylim(0, 0.1)  
    ax.set_xticks(range(xlim_vals..., 5))  
    if k != length(plot_data)
        ax.set_xticklabels([])
    end
    # ax.set_ylabel(@sprintf("\$\\rho_{%i%i}(T, \\omega)\$", idx, idx))
    # ax.legend(frameon=false, handlelength=0, fontsize=12)
    if k%2 == 0
        ax.yaxis.tick_right()
    end
    ax.tick_params(axis="y", labelsize=10)
    ax.legend(frameon=false)
end
# ax = subplot(num_plots, 1, num_plots)
# ax.set_xlabel("\$\\nu\$")
# ax.set_xticklabels(round.(range(xlim_vals..., 7), digits=3))

subplots_adjust(wspace=0.5, hspace=0.16)